In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

In [2]:
X_samples = pickle.load(open('./samples/X_samples.pickle', 'rb')) 
X_test = pickle.load(open('./samples/X_test.pickle', 'rb'))
y_samples = pickle.load(open('./samples/y_samples.pickle', 'rb'))
y_test = pickle.load(open('./samples/y_test.pickle', 'rb'))

In [3]:
# ['Raw', 'SMOTE', 'ADASYN', 'CNN', 'SMOTE + ENN', 'ADASYN + ENN']

In [4]:
print(X_samples.keys())

dict_keys(['Raw', 'SMOTE', 'ADASYN', 'CNN', 'Tomek Links', 'SMOTE + ENN', 'SMOTE + Tomek', 'ADASYN + ENN', 'SVMSMOTE + ENN'])


In [5]:
print(y_samples.keys())

dict_keys(['Raw', 'SMOTE', 'ADASYN', 'CNN', 'Tomek Links', 'SMOTE + ENN', 'SMOTE + Tomek', 'ADASYN + ENN', 'SVMSMOTE + ENN'])


In [6]:
data_list = ['Raw', 'SMOTE', 'ADASYN', 'CNN', 'SMOTE + ENN', 'ADASYN + ENN']

In [7]:
cols = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC_AUC']

In [8]:
X_samples.get(data_list[0])

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
143144,0.005428,-0.540939,0.637584,2.439590,1.316115,0.742650,1.671102,0.280149,0.293844,0.026988,...,-0.055148,-0.270409,-0.238482,-0.338757,-1.002661,-0.060379,-0.276043,-0.001160,-0.093342,-0.296793
258914,0.871545,1.923123,-1.555096,0.211476,-0.174906,-1.760650,0.456333,-1.652432,0.375407,0.892833,...,-0.579051,-0.393833,-0.502683,0.288658,-0.366769,-0.662372,0.528734,0.010908,-0.041962,0.565919
51111,-0.468709,-1.939810,-1.039497,0.429346,-0.198014,2.693946,-2.792994,-0.279832,0.114651,-0.089721,...,0.138139,0.194420,-0.143151,0.354888,0.213072,-0.471635,-0.058909,0.034189,0.240607,-0.296793
146949,0.038652,-0.801246,1.067120,0.506310,-2.533708,0.597024,-1.446026,1.376331,-0.269470,0.208041,...,0.057025,-0.315741,-0.753499,-0.129894,-0.093705,-0.003377,0.381592,0.337806,0.223898,-0.237546
135606,-0.039556,-0.280807,1.109719,0.944761,-0.132693,0.423860,-0.509289,0.698370,-0.020385,-0.333794,...,0.138997,-0.319014,-0.844576,-0.118004,-0.520341,-0.049944,0.124567,0.244372,0.089572,-0.253336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221661,0.680894,-1.196213,1.679891,-0.939175,-1.133975,0.521005,-0.211571,0.487446,0.500638,0.364638,...,0.398187,-0.355852,-0.736666,0.026512,-1.180159,-0.157482,0.214005,0.537133,0.274390,-0.181793
129871,-0.064169,1.175454,-0.001898,1.235163,1.395858,-0.990710,-0.272728,-0.605885,0.103581,0.808439,...,-0.181618,0.113552,0.484107,-0.077394,0.392133,0.488166,-0.239944,0.064726,0.033867,-0.293440
185352,0.492710,0.465324,-3.487401,-4.042365,-0.107532,0.022031,-0.114303,1.643338,-0.613263,-1.390360,...,1.341593,0.437768,-0.345125,-1.040591,-0.302842,0.278728,0.936681,-0.314914,0.057967,13.110878
61180,-0.411119,-0.383532,0.981614,1.235659,0.025899,0.081234,-0.545057,0.540779,0.146734,-0.180970,...,-0.010394,-0.252749,-0.692668,0.032347,-0.016713,-0.282244,0.106386,0.252681,0.091742,-0.255991


In [9]:
y_samples.get(data_list[0])

143144    0
258914    0
51111     0
146949    0
135606    0
         ..
221661    0
129871    0
185352    0
61180     0
88583     0
Name: Class, Length: 227845, dtype: int64

# gdbt : Traditional Gradient Boosting Decision Tree

In [10]:
lgbm = LGBMClassifier()

In [11]:
data_dict = {}
for i in data_list:
    print(i)
    lgbm.fit(X_samples.get(data_list[data_list.index(i)]), y_samples.get(data_list[data_list.index(i)]))
    y_pred = lgbm.predict(X_test)
    y_proba = lgbm.predict_proba(X_test)
    
    acc = accuracy_score(y_pred, y_test)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba[:, 1])
    
    data_dict[i] = [acc, precision, recall, f1, roc_auc]
# print(data_dict)

Raw
SMOTE
ADASYN
CNN
SMOTE + ENN
ADASYN + ENN


In [12]:
df_gbdt = pd.DataFrame(data_dict, index = cols)
df_gbdt

,Raw,SMOTE,ADASYN,CNN,SMOTE + ENN,ADASYN + ENN
Accuracy,0.993136,0.998473,0.997577,0.999561,0.998385,0.997472
Precision,0.013115,0.523179,0.393204,0.915663,0.506329,0.382075
Recall,0.042553,0.840426,0.861702,0.808511,0.851064,0.861702
F1-Score,0.020050,0.644898,0.540000,0.858757,0.634921,0.529412
ROC_AUC,0.518042,0.976024,0.977222,0.968015,0.976519,0.978161


# goss : Gradient-based One-Side Sampling

In [13]:
lgbm = LGBMClassifier(boosting_type = 'goss')

In [14]:
data_dict_param = {}
for i in data_list:
    print(i)
    lgbm.fit(X_samples.get(data_list[data_list.index(i)]), y_samples.get(data_list[data_list.index(i)]))
    y_pred = lgbm.predict(X_test)
    y_proba = lgbm.predict_proba(X_test)
    
    acc = accuracy_score(y_pred, y_test)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba[:, 1])
    
    data_dict_param[i] = [acc, precision, recall, f1, roc_auc]
# print(data_dict)

Raw
SMOTE
ADASYN
CNN
SMOTE + ENN
ADASYN + ENN


In [15]:
df_goss = pd.DataFramee(data_dict_param, index = cols)
df_goss

,Raw,SMOTE,ADASYN,CNN,SMOTE + ENN,ADASYN + ENN
Accuracy,0.993891,0.998560,0.997490,0.999421,0.998508,0.997718
Precision,0.062069,0.541096,0.382775,0.842697,0.529801,0.408163
Recall,0.191489,0.840426,0.851064,0.797872,0.851064,0.851064
F1-Score,0.093750,0.658333,0.528053,0.819672,0.653061,0.551724
ROC_AUC,0.544399,0.977879,0.979515,0.967491,0.977528,0.978566
